<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/Erotica%E7%88%AC%E8%99%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我希望实现一个python爬虫

对于这个界面
https://book.xbookcn.net/p/all.html

里面一个<div class="post-body entry-content" itemprop="description articleBody">的div下，提到了很多链接

如《<a href="https://book.xbookcn.net/search/label/%E6%A2%A6%E4%B8%AD%E7%9A%84%E5%A5%B3%E5%AD%A9" title="梦中的女孩">梦中的女孩</a>》

我希望对这些链接进行抽取，保存为一个list of str，方便我下一步的爬取


In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_links(url):
    # 发送HTTP请求
    response = requests.get(url)
    # 检查请求是否成功
    if response.status_code != 200:
        print("请求失败，状态码：", response.status_code)
        return []

    # 解析HTML内容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 查找包含链接的div
    div = soup.find('div', class_="post-body entry-content")

    # 提取所有的a标签
    links = div.find_all('a') if div else []

    # 获取所有链接的href属性
    hrefs = [link.get('href') for link in links]

    return hrefs

# 目标URL
url = "https://book.xbookcn.net/p/all.html"

# 提取链接
links = extract_links(url)

# 打印提取的链接
# for link in links:
    # print(link)


In [8]:
print(len(links))

296


对于links[0]

是形如

https://book.xbookcn.net/search/label/%E5%A4%9C%E8%89%B2%E4%B8%8B%E7%9A%84%E4%B8%8A%E6%B5%B7%E6%BB%A9

的链接

我希望先建立一个article_name, 把label/之后的编码转换回utf-8（主要是中文字）

寻找所有的形如


<h3 class="post-title entry-title" itemprop="name">
<a href="https://book.xbookcn.net/2000/02/blog-post_2.html">夜色下的上海滩 第二章</a>
</h3>

的元素，保存成一个list of dict,其中包含link和chapter_name两个字段


In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote

def extract_chapters(url):
    # 将URL中的百分比编码转换为UTF-8编码
    article_name = unquote(url.split('/label/')[-1])
    # print("文章名称:", article_name)

    # 发送HTTP请求
    response = requests.get(url)
    # 检查请求是否成功
    if response.status_code != 200:
        print("请求失败，状态码：", response.status_code)
        return []

    # 解析HTML内容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 查找所有符合条件的h3标签
    h3_tags = soup.find_all('h3', class_="post-title entry-title")

    # 提取链接和章节名称
    chapters = []
    for tag in h3_tags:
        a_tag = tag.find('a')
        if a_tag:
            chapters.append({
                'link': a_tag.get('href'),
                'chapter_name': a_tag.get_text().strip()
            })

    return chapters, article_name

# 示例链接
chapters,article_name = extract_chapters(links[1])


In [15]:
datas = []

from tqdm import tqdm
import time

for link in tqdm(links):
    try:
        chapters,article_name = extract_chapters(link)
        datas.append({
            'article_name': article_name,
            'chapters': chapters
        })
        time.sleep(0.3)
    except Exception as e:
        print(e)

  0%|          | 0/296 [00:00<?, ?it/s]

Invalid URL '/p/popular.html': No scheme supplied. Perhaps you meant https:///p/popular.html?


 12%|█▎        | 37/296 [00:22<02:35,  1.67it/s]

Invalid URL '/p/urban.html': No scheme supplied. Perhaps you meant https:///p/urban.html?


 32%|███▏      | 94/296 [00:59<02:21,  1.43it/s]

...

 87%|████████▋ | 258/296 [02:45<00:22,  1.72it/s]

Invalid URL '/p/dark.html': No scheme supplied. Perhaps you meant https:///p/dark.html?


 94%|█████████▍| 279/296 [02:59<00:11,  1.48it/s]

Invalid URL '/p/romance.html': No scheme supplied. Perhaps you meant https:///p/romance.html?


100%|██████████| 296/296 [03:10<00:00,  1.56it/s]


In [16]:
import pickle

# datas = [1, 2, 3] # 示例数据

with open('/content/link_datas.pkl', 'wb') as f:
    pickle.dump(datas, f)

In [ ]:
for i, data in  enumerate(datas):
    print(data['article_name'],end = ' ')
    if (i+1) % 7 == 0:
        print()

In [20]:
print(datas[1])

{'article_name': '舌战法庭', 'chapters': [{'link': 'https://book.xbookcn.net/2000/02/blog-post_786.html', 'chapter_name': '舌战法庭 内容简介'}, {'link': 'https://book.xbookcn.net/2000/02/blog-post_1.html', 'chapter_name': '舌战法庭 第一章'}, {'link': 'https://book.xbookcn.net/2000/02/blog-post_22.html', 'chapter_name': '舌战法庭 第二章'}, {'link': 'https://book.xbookcn.net/2000/02/blog-post_18.html', 'chapter_name': '舌战法庭 第三章'}, {'link': 'https://book.xbookcn.net/2000/02/blog-post_59.html', 'chapter_name': '舌战法庭 第四章'}, {'link': 'https://book.xbookcn.net/2000/02/blog-post_48.html', 'chapter_name': '舌战法庭 第五章'}, {'link': 'https://book.xbookcn.net/2000/02/blog-post_65.html', 'chapter_name': '舌战法庭 第六章'}, {'link': 'https://book.xbookcn.net/2000/02/blog-post_79.html', 'chapter_name': '舌战法庭 第七章'}]}


In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_content(link):
    # 发送HTTP请求
    response = requests.get(link)
    # 检查请求是否成功
    if response.status_code != 200:
        print("请求失败，状态码：", response.status_code)
        return ""

    # 解析HTML内容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 查找所有具有特定类名的div元素
    divs = soup.find_all('div', class_="post-body entry-content")

    if len(divs)>1:
        print("警告：找到多个div：")

    # 检查itemprop属性并选择内容最长的div
    longest_content = ""

    for i, div in enumerate(divs):


        content = div.get_text()
        if len(content) > len(longest_content):
            longest_content = content

    if divs[i].get('itemprop') != "description articleBody":
            print("警告：找到不符合条件的div：")
            print(str(div)[:200])  # 打印div的前200个字符

    return longest_content

# 示例链接
links = ["https://book.xbookcn.net/2003/08/blog-post_229.html", "https://book.xbookcn.net/2000/02/blog-post_1.html"]

# 提取内容
for link in links:
    content = extract_content(link)
    print("提取的内容：", content[:200])  # 打印内容的前200个字符



我已经验证 extract_content(link) 可以使用


father_folder = "/content/output"

for i, data in enumerate(datas):
    chapters = data['chapters']
    article_name = data['article_name']
    
    # 在father_folder下建立以article_name明明的文件夹

    for j, chapter in enumerate(chapters[1:]):
        text = extract_content(chapter['link'])
        if text.strip() == "":
            continue
        # fname = father_folder + "chapter_" + str(j) + ".txt"

        # 保存text到fname中

        
我希望进一步补全下面的代码，对于每个article，建立子文件夹

然后爬取text保存到对应的子文件夹中


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
!ls /content/drive/MyDrive/Wuxia/erotica

In [ ]:
father_folder = "/content/drive/MyDrive/Wuxia/erotica"

import os
import requests
from bs4 import BeautifulSoup

# 假设extract_content函数已经定义
# def extract_content(link):
#     ...

for data in tqdm(datas):
    chapters = data['chapters']
    article_name = data['article_name']

    # 创建以article_name命名的文件夹
    article_folder = os.path.join(father_folder, article_name)
    if not os.path.exists(article_folder):
        os.makedirs(article_folder)

    for j, chapter in enumerate(chapters[1:]):
        text = extract_content(chapter['link'])
        if text.strip() == "":
            continue

        # 创建文件名
        fname = os.path.join(article_folder, f"chapter_{j}.txt")

        # 保存text到文件中
        with open(fname, 'w', encoding='utf-8') as file:
            file.write(text)

        # print(f"已保存：{fname}")

        time.sleep(0.1)


 11%|█         | 32/288 [04:59<45:25, 10.64s/it]